# EDA steps:
•	Import the Data-set
•	slicing data
•	Check data characters
•	calculating statistics: mean, median, standard deviation
•	basic plotting for data
•	Correlation computation

Data pre-processing steps:
•	Check out and filled the missing values
•	Import the Libraries
•   Read the bureau data
•	Handle /Create Dummy variable
•   Group by function
•   Logistic Regrssion
•	Dimensionality reduction
•	Clustering of similar observations

In [2]:
# Import the Data-set & Import the Libraries

In [3]:
# numpy and pandas for data manipulation
import pandas as pd
import numpy as np
# matplotlib and seaborn for plotting
import matplotlib
import matplotlib.pyplot as plt 
import seaborn as sns 

Read Data

In [5]:
app_train = pd.read_csv("/dbfs/FileStore/tables/application_train.csv", header='infer')
app_test = pd.read_csv("/dbfs/FileStore/tables/application_test.csv", header='infer')
bureau = pd.read_csv("/dbfs/FileStore/tables/bureau.csv", header='infer')
bureau_balance = pd.read_csv("/dbfs/FileStore/tables/bureau_balance.csv", header='infer')


Slicing data

Check data characters & calculating statistics: mean, median, standard deviation

In [8]:
app_train['TARGET'].value_counts()

In [9]:
print('AMT_INCOME_TOTAL Train',app_train['AMT_INCOME_TOTAL'].count())

basic plotting for data

In [11]:
app_train['TARGET'].astype(int).plot.hist()

Correlation computation

In [13]:
# Find correlations with the target and sort
correlations = app_train.corr()['TARGET'].sort_values()

# Display correlations
print('Most Positive Correlations:\n', correlations.tail(20))
print('\nMost Negative Correlations:\n', correlations.head(20))

Correlation Heat Map

In [15]:
ext_data = app_train[['TARGET', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'DAYS_BIRTH']]
ext_data_corrs = ext_data.corr()
ext_data_corrs

plt.figure(figsize = (8, 6))

# Heatmap of correlations
sns.heatmap(ext_data_corrs, cmap = plt.cm.RdYlBu_r, vmin = -0.25, annot = True, vmax = 0.6)
plt.title('Correlation Heatmap');

Check out and filled the missing values

In [17]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [18]:
 # Missing values statistics
missing_values = missing_values_table(app_train)
missing_values.head(20)

Exploration of Bureau Data

Import the Libraries

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Imputer 
from sklearn.preprocessing import scale 
# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

Group By Function

In [23]:
# Groupby the client id (SK_ID_CURR), count the number of previous loans, and rename the column
previous_loan_counts = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})
previous_loan_counts.head()

Out[ 206 ]: 
 SK_ID_CURR previous_loan_counts
0 100001 7
1 100002 8
2 100003 4
3 100004 2
4 100005 3

In [24]:
# app_train left join previous loan data
train = app_train.join(previous_loan_counts, on = 'SK_ID_CURR', how = 'left', lsuffix='SK_ID_CURR', rsuffix='r_SK_ID_CURR')

# Fill the missing values with 0 
train['previous_loan_counts'] = train['previous_loan_counts'].fillna(0)
train.head()

Out[ 207 ]: 
 SK_ID_CURRSK_ID_CURR TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR \
0 100002 1 Cash loans M N 
1 100003 0 Cash loans F N 
2 100004 0 Revolving loans M Y 
3 100006 0 Cash loans F N 
4 100007 0 Cash loans M N 

 FLAG_OWN_REALTY CNT_CHILDREN AMT_INCOME_TOTAL AMT_CREDIT AMT_ANNUITY \
0 Y 0 202500.0 406597.5 24700.5 
1 N 0 270000.0 1293502.5 35698.5 
2 Y 0 67500.0 135000.0 6750.0 
3 Y 0 135000.0 312682.5 29686.5 
4 Y 0 121500.0 513000.0 21865.5 

 ... FLAG_DOCUMENT_20 FLAG_DOCUMENT_21 \
0 ... 0 0 
1 ... 0 0 
2 ... 0 0 
3 ... 0 0 
4 ... 0 0 

 AMT_REQ_CREDIT_BUREAU_HOUR AMT_REQ_CREDIT_BUREAU_DAY \
0 0.0 0.0 
1 0.0 0.0 
2 0.0 0.0 
3 NaN NaN 
4 0.0 0.0 

 AMT_REQ_CREDIT_BUREAU_WEEK AMT_REQ_CREDIT_BUREAU_MON \
0 0.0 0.0 
1 0.0 0.0 
2 0.0 0.0 
3 NaN NaN 
4 0.0 0.0 

 AMT_REQ_CREDIT_BUREAU_QRT AMT_REQ_CREDIT_BUREAU_YEAR \
0 0.0 1.0 
1 0.0 0.0 
2 0.0 0.0 
3 NaN NaN 
4 0.0 0.0 

 SK_ID_CURRr_SK_ID_CURR previous_loan_counts 
0 216379.0 1.0 
1 216380.0 8.0 
2 216381.0 4.0 
3 216383.0 2.0 
4 216384.0 2.0 

[5 rows x 124 columns]

In [25]:
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)
print('Training Features shape: ', train.shape)


Training Features shape: (307511, 124)

In [26]:
train_labels = app_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# Add the target back in
app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape: (307511, 243)
Testing Features shape: (48744, 242)

In [27]:
# Logistic Regression
from sklearn.preprocessing import MinMaxScaler, Imputer
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

# Match the columns in the dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
print('Training shape: ', app_train.shape)
print('Testing shape: ', app_test.shape)


Training shape: (307511, 242)
Testing shape: (48744, 242)

In [28]:
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

print('Training set full shape: ', app_train.shape)
print('Testing set full shape: ' , app_test.shape)

Training set full shape: (307511, 242)
Testing set full shape: (48744, 242)

In [29]:
# Copy of the testing data
app_test = app_test.copy()

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(app_train)

# Transform both training and testing data
app_train = imputer.transform(app_train)
app_test = imputer.transform(app_test)

# Repeat with the scaler
scaler.fit(app_train)
app_train = scaler.transform(app_train)
app_test = scaler.transform(app_test)

print('Training data shape: ', app_train.shape)
print('Testing data shape: ', app_test.shape)

Training data shape: (307511, 242)
Testing data shape: (48744, 242)

In [30]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(app_train, train_labels)

Out[ 216 ]: 
LogisticRegression(C=0.0001, class_weight=None, dual=False,
 fit_intercept=True, intercept_scaling=1, max_iter=100,
 multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
 solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
app_train = pd.read_csv("/dbfs/FileStore/tables/application_train.csv", header='infer')
app_test = pd.read_csv("/dbfs/FileStore/tables/application_test.csv", header='infer')

In [32]:
# Naive Bayes
app_train = pd.get_dummies(app_train)
# Match the columns in the dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
print('Training shape: ', app_train.shape)
print('Testing shape: ', app_test.shape)

In [33]:
# Split dataset in training and test datasets
X_train, X_test = train_test_split(app_train, test_size=0.3, random_state = 0)

# fit a Naive Bayes model to the data
model = GaussianNB()
model.fit(X_train, X_test)
print(model)
# make predictions
expected = X_test
predicted = model.predict(X_train)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

In [34]:
# Decision Tree Classifier
from sklearn import datasets
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

# load the datasets
app_train = pd.read_csv("/dbfs/FileStore/tables/application_train.csv", header='infer')
app_train = pd.get_dummies(app_train)
app_train.dropna()

# Split dataset in training and test datasets
X_train, X_test = train_test_split(app_train, test_size=0.3, random_state = 0)

# fit a CART model to the data
model = DecisionTreeClassifier()
model.fit(X_train, X_test)
print(model)

# make predictions
expected = X_test.target
predicted = model.predict(X_train)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

In [35]:
# Support Vector Machine
from sklearn import datasets
from sklearn import metrics
from sklearn.svm import SVC
# load the datasets
app_train = pd.read_csv("/dbfs/FileStore/tables/application_train.csv", header='infer')
app_train = pd.get_dummies(app_train)
app_train.dropna()
# Split dataset in training and test datasets
X_train, X_test = train_test_split(app_train, test_size=0.3, random_state = 0)
# fit a SVM model to the data
model = SVC()
model.fit(X_train, X_test)
print(model)
# make predictions
expected = X_test
predicted = model.predict(X_train)
# summarize the fit of the model
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

In [36]:
# Random forest
from sklearn.ensemble import RandomForestClassifier

# Make the random forest classifier
random_forest = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)


In [37]:
random_forest.fit(app_train, train_labels)

# Extract feature importances
feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})

# Make predictions on the test data
predictions = random_forest.predict_proba(app_test)[:, 1]

[Parallel(n_jobs=-1)]: Done 34 tasks | elapsed: 20.3s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 54.1s finished
[Parallel(n_jobs=8)]: Done 34 tasks | elapsed: 0.4s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed: 1.0s finished

In [38]:
# AdaBoost
from sklearn.ensemble import AdaBoostClassifier #For Classification
from sklearn.ensemble import AdaBoostRegressor #For Regression
from sklearn.tree import DecisionTreeClassifier

# load the datasets
app_train = pd.read_csv("/dbfs/FileStore/tables/application_train.csv", header='infer')
app_train = pd.get_dummies(app_train)
app_train.dropna(how="all", inplace=True)

# Split dataset in training and test datasets
app_train_train, app_train_test, X, y = train_test_split(app_train, predict, test_size = 0.25, random_state = 42)
dt = DecisionTreeClassifier() 
clf = AdaBoostClassifier(n_estimators=100, base_estimator=dt,learning_rate=1)

# Above I have used decision tree as a base estimator, you can use any ML learner as base estimator if it ac# cepts sample weight 
clf.fit(app_train, predict)

In [39]:
#PCA

In [40]:
#PCA
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# matplotlit and seaborn for visualizations
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 22
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA
from sklearn.preprocessing import Imputer
from sklearn.pipeline import Pipeline

app_train = pd.read_csv("/dbfs/FileStore/tables/application_train.csv", header='infer')
app_test = pd.read_csv("/dbfs/FileStore/tables/application_test.csv", header='infer')

In [41]:
# Train missing values (in percent)
app_train_missing = (app_train.isnull().sum() / len(app_train)).sort_values(ascending = False)
app_train_missing.head()

Out[ 221 ]: 
COMMONAREA_MEDI 0.698723
COMMONAREA_AVG 0.698723
COMMONAREA_MODE 0.698723
NONLIVINGAPARTMENTS_MODE 0.694330
NONLIVINGAPARTMENTS_MEDI 0.694330
dtype: float64

In [42]:
# Test missing values (in percent)
app_test_missing = (app_test.isnull().sum() / len(app_test)).sort_values(ascending = False)
app_test_missing.head()

Out[ 222 ]: 
COMMONAREA_MEDI 0.687161
COMMONAREA_AVG 0.687161
COMMONAREA_MODE 0.687161
NONLIVINGAPARTMENTS_MODE 0.684125
NONLIVINGAPARTMENTS_MEDI 0.684125
dtype: float64

In [43]:
 # One hot encoding
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

# Match the columns in the dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)
print('Training shape: ', app_train.shape)
print('Testing shape: ', app_test.shape)


Training shape: (307511, 242)
Testing shape: (48744, 242)

In [44]:
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

print('Training set full shape: ', app_train.shape)
print('Testing set full shape: ' , app_test.shape)

Training set full shape: (307511, 242)
Testing set full shape: (48744, 242)

In [45]:
# Make a pipeline with imputation and pca
pipeline = Pipeline(steps = [('imputer', Imputer(strategy = 'median')),
             ('pca', PCA())])

# Fit and transform on the training data
app_train_pca = pipeline.fit_transform(app_train)

# transform the testing data
app_test_pca = pipeline.transform(app_test)

In [46]:
# Extract the pca object
pca = pipeline.named_steps['pca']

# Plot the cumulative variance explained

plt.figure(figsize = (10, 8))
plt.plot(list(range(app_train.shape[1])), np.cumsum(pca.explained_variance_ratio_), 'r-')
plt.xlabel('Number of PC'); plt.ylabel('Cumulative Variance Explained');
plt.title('Cumulative Variance Explained with PCA');